<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <br/>
    <font>Python 2024</font><br/>
    <br/>
    <br/>
    <b style="font-size: 1.5em">дескрипторы и метаклассы</b><br/>
    <br/>
    <font>Денис Макаров</font><br/>
</center>

In [32]:
class Cow:
    def __init__(self, name: str) -> None:
        self.set_name(name)

    # Хотим валидировать, чтобы имя было непустой строкой из символов
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError(f"Expected type <str> for name, got <{type(name).__name__}>")
        if not name or not name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name
        
cow = Cow("Isabella")
for name in (1, "", "Ja Ra"):
    try:
        cow.set_name(name)
    except ValueError as e:
        print(e)

Expected type <str> for name, got <int>
Name should be non-empty alphanumeric string
Name should be non-empty alphanumeric string


In [2]:
class Sheep:
    def __init__(self, name: str):
        self.set_name(name)
        
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError(f"Expected type <str> for name, got <{type(name).__name__}>")
        if not name or not name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name

## Решения

### Наследование

In [3]:
class Animal:
    def __init__(self, name: str) -> None:
        self.set_name(name)
        
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError(f"Expected type <str> for name, got <{type(name).__name__}>")
        if not name or not name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name
        
class Cow(Animal):
    pass

class Sheep(Animal):
    pass

cow = Cow("Isabella")
sheep = Sheep("Boris")
for animal in (cow, sheep):
    try:
        cow.set_name("")
    except ValueError as e:
        print(e)

Name should be non-empty alphanumeric string
Name should be non-empty alphanumeric string


### Проблема

In [4]:
class Farmer:
    def __init__(self, name: str, surname: str) -> None:
        self.set_name(name)
        self.set_surname(surname)
        
    def set_name(self, name: str) -> None:
        if not isinstance(name, str):
            raise ValueError(f"Expected type <str> for name, got <{type(name).__name__}>")
        if not name or name.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._name = name
        
    def set_surname(self, surname: str) -> None:
        if not isinstance(surname, str):
            raise ValueError(f"Expected type <str> for name, got <{type(name).__name__}>")
        if not surname or not surname.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._surname = surname

In [5]:
class Farmer(Animal):  # <- казус
    def __init__(self, name: str, surname: str) -> None:
        super().__init__(name)
        self.set_surname(surname)
        
    def set_surname(self, surname: str) -> None:
        if not isinstance(surname, str):
            raise ValueError(f"Expected type <str> for name, got <{type(name).__name__}>")
        if not surname or not surname.isalnum():
            raise ValueError("Name should be non-empty alphanumeric string")
        self._surname = surname

## Дескрипторы
https://docs.python.org/3/howto/descriptor.html

In [6]:
class Descriptor:
    # Any of this methods -> it is descriptor
    def __get__(self, obj, objtype):
        print(f"Descriptor.__get__(self={self}, obj={obj}, objtype={objtype})")

    def __set__(self, obj, value):
        print(f"Descriptor.__set__(self={self}, obj={obj}, value={value})")

    def __delete__(self, obj):
        print(f"Descriptor.__delete__(self={self}, obj={obj})")

class Class:
    attr = Descriptor()

class_object = Class()
class_object.attr = 10  # __set__ is called
class_object.attr       # __get__ is called
del class_object.attr   # __delete__ is called

Descriptor.__set__(self=<__main__.Descriptor object at 0x105f1fce0>, obj=<__main__.Class object at 0x105f1ed80>, value=10)
Descriptor.__get__(self=<__main__.Descriptor object at 0x105f1fce0>, obj=<__main__.Class object at 0x105f1ed80>, objtype=<class '__main__.Class'>)
Descriptor.__delete__(self=<__main__.Descriptor object at 0x105f1fce0>, obj=<__main__.Class object at 0x105f1ed80>)


### Валидация с помощью дескриптора

In [8]:
import typing as t

class NonEmptyString:
    
    def __init__(self, name: str):
        self._name = name
    
    def __get__(self, obj: t.Any | None, objtype: type) -> t.Any:
        return getattr(obj, self._name)
    
    def __set__(self, obj: t.Any, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError("Expected type <str> for value, got <{}>".format(type(value).__name__))
        if not value or not value.isalnum():
            raise ValueError("Value should be non-empty alphanumeric string")
        setattr(obj, self._name, value)
        
    def __delete__(self, obj: t.Any) -> None:
        raise ValueError("Value cannot be deleted")

In [9]:
class Farmer:
    name = NonEmptyString("_name")
    surname = NonEmptyString("_surname")
    
    def __init__(self, name: str, surname: str):
        self.name = name
        self.surname = surname
    

farmer = Farmer("Leo", "Pellegro")
print(farmer.name)
farmer.name = "Nick"
print(farmer.name)

Leo
Nick


In [10]:
try:
    farmer.name = ""
except ValueError as e:
    print(e)

Value should be non-empty alphanumeric string


In [11]:
try:
    del farmer.name
except ValueError as e:
    print(e)

Value cannot be deleted


### `__set_name__`

In [12]:
# Python 3.6

class NonEmptyString2:
    
    def __set_name__(self, owner: t.Any, name: str):
        self._name = f"_{name}"

    def __get__(self, obj: t.Any | None, objtype: type) -> t.Any:
        return getattr(obj, self._name)

    def __set__(self, obj: t.Any, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError("Expected type <str> for value, got <{}>".format(type(value).__name__))
        if not value or not value.isalnum():
            raise ValueError("Value should be non-empty alphanumeric string")
        setattr(obj, self._name, value)

    def __delete__(self, obj: t.Any) -> None:
        raise ValueError("Value cannot be deleted")

In [13]:
class Farmer:
    name = NonEmptyString2()
    surname = NonEmptyString2()
    
    def __init__(self, name: str, surname: str) -> None:
        self.name = name
        self.surname = surname

farmer = Farmer("Nikolai", "Jos")
print(farmer.name, farmer.surname)
print(farmer.__dict__)

Nikolai Jos
{'_name': 'Nikolai', '_surname': 'Jos'}


### Non-data descriptors vs data descriptors

In [14]:
# only get
class NonDataDescriptor:
    def __get__(self, obj, objtype):
        pass

# set or delete

class DataDescriptor:
    def __set__(self, obj, value):
        pass

class AlsoDataDescriptor:
    def __delete__(self, obj):
        pass

### Порядок доступа к аттрибуту

`object.__getattribute__`

In [15]:
def find_name_in_mro(cls, name, default):
    "Emulate _PyType_Lookup() in Objects/typeobject.c"
    for base in cls.__mro__:
        if name in vars(base):
            return vars(base)[name]
    return default

def object_getattribute(obj, name):
    "Emulate PyObject_GenericGetAttr() in Objects/object.c"
    null = object()
    objtype = type(obj)
    cls_var = find_name_in_mro(objtype, name, null)
    descr_get = getattr(type(cls_var), '__get__', null)
    if descr_get is not null:
        if (hasattr(type(cls_var), '__set__')
            or hasattr(type(cls_var), '__delete__')):
            return descr_get(cls_var, obj, objtype)     # data descriptor
    if hasattr(obj, '__dict__') and name in vars(obj):
        return vars(obj)[name]                          # instance variable
    if descr_get is not null:
        return descr_get(cls_var, obj, objtype)         # non-data descriptor
    if cls_var is not null:
        return cls_var                                  # class variable
    raise AttributeError(name)

### `__getattribute__` vs `__getattr__`

In [16]:
def getattr_hook(obj, name):
    "Emulate slot_tp_getattr_hook() in Objects/typeobject.c"
    try:
        return obj.__getattribute__(name)
    except AttributeError:
        if not hasattr(type(obj), '__getattr__'):
            raise
    return type(obj).__getattr__(obj, name)

### Property

In [17]:
# https://docs.python.org/3/howto/descriptor.html#properties

class Animal:
    def __init__(self, usd_price: float) -> None:
        self._usd_price = usd_price
        
    def get_price(self) -> float:
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub
    
    def set_price(self, price: float) -> None:
        if price <= 0:
            raise ValueError("Price must be positive, got {}".format(price))
        self._usd_price = price

In [18]:
animal = Animal(100)
print(animal.get_price())
animal.set_price(10)
print(animal.get_price())
try:
    animal.set_price(-10)
except ValueError as e:
    print(e)

6700
670
Price must be positive, got -10


In [19]:
class Animal:
    def __init__(self, usd_price):
        self._usd_price = usd_price
        
    @property
    def price(self):
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub

In [27]:
animal = Animal(100)
print(animal.price)

try:
    animal.price = 10
except AttributeError as e:
    print(e)

6700
property 'price' of 'Animal' object has no setter


In [28]:
class Animal:
    def __init__(self, usd_price):
        self._usd_price = usd_price
        
    @property
    def price(self):
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub
    
    @price.setter
    def price(self, price):
        if price <= 0:
            raise ValueError("Price must be positive, got {}".format(price))
        self._usd_price = price

In [29]:
animal = Animal(100)
print(animal.price)
animal.price = 10
print(animal.price)
try:
    animal.price = -10
except ValueError as e:
    print(e)

6700
670
Price must be positive, got -10


In [30]:
class Animal:
    def __init__(self, usd_price: float) -> None:
        self._usd_price = usd_price
        
    def get_price(self) -> float:
        usd_to_rub = 67  # сходить за актуальным курсом usd
        return self._usd_price * usd_to_rub
    
    def set_price(self, price: float) -> None:
        if price <= 0:
            raise ValueError("Price must be positive, got {}".format(price))
        self._usd_price = price
        
    price = property(get_price, set_price)

In [31]:
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc

    def __set_name__(self, owner, name):
        self.__name__ = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError
        self.fdel(obj)

    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)

Тоже дескрипторы или ведут себя как дескрипторы:
1) methods, `staticmethod`, `classmethod`
2) `django.db.models.Field`, `sqlalchemy.Column`
3) `dataclasses.field`  

## Вопросы?

## Создание объектов

### Инициализация

In [ ]:
class Class1:      
    def __init__(self, x: int) -> None:
        print(f"Class1.__init__({x})")
        self.x = x
        
a = Class1(1)
print(type(a))
print(a.x)

In [34]:
class Class1:      
    def __init__(self, x: int) -> None:
        print(f"Class1.__init__({x})")
        self.x = x
        
a = Class1(1)
print(type(a))
print(a.x)

Class1.__init__(1)
<class '__main__.Class1'>
1


![__init__](meta/img/img.001.jpeg "__init__")

### Выделение памяти под объект

In [ ]:
class Class:
    pass

In [ ]:
c = Class()

In [ ]:
import dis

def f():
    return Class()
    
dis.dis(f)

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    ???

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    # выделить место под объект
    # инициализировать объект
    # вернуть готовый объект

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    # выделить место под объект
    obj = ???
    # инициализировать объект
    Class.__init__(obj, *args, **kwargs)
    # вернуть готовый объект
    return obj

In [ ]:
# kindof
def Class(*args, **kwargs) -> Class:
    # выделить место под объект
    obj = Class.__new__(*args, **kwargs)
    # инициализировать объект
    Class.__init__(obj, *args, **kwargs)
    # вернуть готовый объект
    return obj

In [ ]:
class Class2:
    def __new__(cls, *args) -> "Class2":
        print(f"Class2.__new__(cls={cls}, args={args})")
        return object.__new__(cls)
    
    def __init__(self, *args) -> None:
        print(f"Class2.__init__({args})")
        
c = Class2(2)
print(type(c))

In [35]:
class Class2:
    def __new__(cls, *args) -> "Class2":
        print(f"Class2.__new__(cls={cls}, args={args})")
        return object.__new__(cls)
    
    def __init__(self, *args) -> None:
        print(f"Class2.__init__({args})")
        
c = Class2(2)
print(type(c))

Class2.__new__(cls=<class '__main__.Class2'>, args=(2,))
Class2.__init__((2,))
<class '__main__.Class2'>


In [36]:
c = Class2(1)
print("-")
c = Class2.__new__(Class2, 1)
c.__init__(1)

Class2.__new__(cls=<class '__main__.Class2'>, args=(1,))
Class2.__init__((1,))
-
Class2.__new__(cls=<class '__main__.Class2'>, args=(1,))
Class2.__init__((1,))


In [ ]:
# Забавная штука
class Class2:
    def __new__(cls, *args) -> int:
        return 1
    
    def __init__(self, *args) -> None:
        self.x = args[0]
        
x = Class2()
print(type(x))
print(x.x)

In [37]:
# Забавная штука
class Class2:
    def __new__(cls, *args) -> int:
        return 1
    
    def __init__(self, *args) -> None:
        self.x = args[0]
        
x = Class2()
print(type(x))
print(x.x)

<class 'int'>


AttributeError: 'int' object has no attribute 'x'

In [ ]:
c = Class2(1)
print("-")
c = Class2.__new__(Class2, 1)
if isinstance(c, Class2):
    c.__init__(1)

![__new__](meta/img/img.002.jpeg "__new__")

### Загадка

In [40]:
class Class:
    pass

def func():
    pass

print(Class.__dict__)
print(func.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Class' objects>, '__weakref__': <attribute '__weakref__' of 'Class' objects>, '__doc__': None}
{}


In [41]:
class Class:
    @classmethod
    def __call__(cls):
        raise RuntimeError()

x = Class()
x

In [42]:
x()

RuntimeError: 

In [43]:
# https://docs.python.org/3/library/functions.html#type

class X:
    a = 1

X = type("X", (object, ), dict(a=1))

In [44]:
Class = type("Class", (object, ), dict())
print(Class.__call__)
print(Class.__call__())

<method-wrapper '__call__' of type object at 0x109e1b260>


In [33]:
def class_call(self):
    raise RuntimeError()

Class = type("Class", (object, ), {"__call__": class_call})
x = Class()
x()

RuntimeError: 

### PyTypeObject

```c++
struct PyTypeObject {
    PyObject* tp_name;
    PyObject* tp_new;
    PyObject* tp_init;
    PyObject* tp_call;
    PyObject* tp_dict;
};
```

In [ ]:
class X:
    def __new__(...):
        pass
    
    def __init__(...):
        pass
    
    def __call__(...):
        pass

```c++
PyTypeObject PyType_X;
PyType_X.tp_name = "X";
PyType_X.tp_new = __new__
PyType_X.tp_init = __init__
PyType_X.tp_call = type_call
PyType_X.tp_dict = {"__call__": __call__}
```

```c++
// Objects/typeobject.c
// very distilled
static PyObject *
type_call(PyTypeObject *type, PyObject *args, PyObject *kwds)
{
    PyObject *obj;

    if (type->tp_new == NULL) {
        // assert
    }

    obj = type->tp_new(type, args, kwds);
    if (obj == NULL)
        return NULL;
    
    if (!PyType_IsSubtype(Py_TYPE(obj), type))
        return obj;

    if (type->tp_init != NULL) {
        int res = type->tp_init(obj, args, kwds);
        // assert if res < 0
    }
    return obj;
}
```

### Почему X() всегда работает?

In [ ]:
X()

```c++
X.tp_call()
// X.tp_call == type_call
```

### Итого

__Python__
```python
class X:
    a = 1
    
    def __new__(...):
        pass
    
    def __init__(...):
        pass
```

__C__
```c++
PyTypeObject PyType_X;
PyType_X.tp_name = "X";
PyType_X.tp_new = __new__
PyType_X.tp_init = __init__
PyType_X.tp_call = type_call
PyType_X.tp_dict = {"a": 1}


PyObject* type_call() {
    PyObject* x = PyType_X.tp_new();
    PyType_X.tp_init(x);
    return x
}
```

### metaclass.\_\_call__

In [35]:
class Class3:
    def __new__(cls, *args, **kwargs) -> t.Self:
        print(f"Class3.__new__({cls}, {args}, {kwargs})")
        return object.__new__(cls)
    
    def __init__(
        self, 
        *args, 
        **kwargs
    ) -> None:
        print(f"Class3.__init__({self}, {args}, {kwargs})")

In [36]:
class MetaclassA(type):
    def __call__(cls, *args, **kwargs) -> "Class3":
        print(f"MetaclassA.__call__({cls}, {args}, {kwargs})")
        obj = cls.__new__(cls, *args, **kwargs)
        obj.__init__(*args, **kwargs)
        return obj
        
class Class3(metaclass=MetaclassA):
    def __new__(cls, *args, **kwargs) -> t.Self:
        print(f"Class3.__new__({cls}, {args}, {kwargs})")
        return object.__new__(cls)
    
    def __init__(self, *args, **kwargs) -> None:
        print(f"Class3.__init__({self}, {args}, {kwargs})") 
        
x = Class3(1)
print(x)

MetaclassA.__call__(<class '__main__.Class3'>, (1,), {})
Class3.__new__(<class '__main__.Class3'>, (1,), {})
Class3.__init__(<__main__.Class3 object at 0x1039dc650>, (1,), {})


In [37]:
# Class3 = type("Class3", (object, ), dict())
Class3 = MetaclassA("Class3", (object, ), dict())

In [38]:
Class3()

MetaclassA.__call__(<class '__main__.Class3'>, (), {})


```c++
PyType_Class3.tp_call != type_call
PyType_Class3.tp_call == MetaclassA.__call__
type_call == type.tp_call
```

![__call__](meta/img/img.003.jpeg "__call__")

```c++
PyObject *
_PyObject_New(PyTypeObject *tp)
{
    PyObject *op = (PyObject *) PyObject_MALLOC(_PyObject_SIZE(tp));
    if (op == NULL) {
        return PyErr_NoMemory();
    }
    _PyObject_Init(op, tp);
    return op;
}

static inline void
_PyObject_Init(PyObject *op, PyTypeObject *typeobj)
{
    assert(op != NULL);
    Py_SET_TYPE(op, typeobj);
    if (_PyType_HasFeature(typeobj, Py_TPFLAGS_HEAPTYPE)) {
        Py_INCREF(typeobj);
    }
    _Py_NewReference(op);
}
```

### Flyweight

https://sourcemaking.com/design_patterns/flyweight

In [ ]:
class Image:
    def __init__(self, path: str):
        self._path = path
    
    def __repr__(self):
        return "Image(path={})".format(self._path)

In [ ]:
class Image:
    def __init__(self, path: str):
        # load image to ram
        # slow and ram consuming

        
class Tree:
    def __init__(self, x: int, y: int, path: str):
        self._x = x
        self._y = y
        self._image = Image(path)

In [ ]:
def create_forest() -> list[Tree]:
    trees = []
    for i in range(100):
        path = "imgs/tree{}.png".format(i % 3)
        trees.append(Tree(x, 10, path))

In [ ]:
class Tree:
    def __init__(self, x: int, y: int, path: str | None = None, image: Image | None = None):
        if image is not None:
            self._image = image
        elif path is not None:
            self._image = Image(path)
        else:
            raise RuntimeError("Either path or image must be specified")
        self._x = x
        self._y = y
        
image_lib = ImageLibrary()
tree = Tree(x, y, image=ImageLibrary.get("imgs/tree.png"))

In [ ]:
class Flyweight(type):
    IMAGES = {}
    
    def __call__(cls, x: int, y: int, path: str | None = None, image: Image | None = None) -> "Tree":
        obj = cls.__new__(cls)
        if path is not None:
            if path in Flyweight.IMAGES:
                image = Flyweight.IMAGES[path]
            else:
                image = Image(path)
                Flyweight.IMAGES[path] = image
        cls.__init__(obj, x, y, path=path, image=image)
        return obj
    
class Tree(metaclass=Flyweight):
    def __init__(self, x: int, y: int, path: str | None = None, image: Image | None = None):
        if image is not None:
            self._image = image
        elif path is not None:
            self._image = Image(path)
        else:
            raise RuntimeError("Either path or image must be specified")
        self._x = x
        self._y = y

In [ ]:
tree1 = Tree(1, 2, "tree.png")
tree2 = Tree(2, 3, "tree.png")
assert tree1._image is tree2._image

### type()

In [39]:
class X:
    a = 1
    
X = type("X", (object, ), dict(a=1))

In [ ]:
class type:
    @classmethod
    def __call__(...):
        # smth

X = type.__call__("X", (object, ), dict(a=1))

In [ ]:
class type:
    @classmethod
    def __call__(...):
        # smth
        
type = type("type", (object, ), {"__call__": ...})

```c++
PyTypeObject PyType_Type;
PyType_Type.tp_name = "type";
PyType_Type.tp_new = type_new
PyType_Type.tp_init = type_init
PyType_Type.tp_call = type_call
```

In [41]:
class Class:
    def __init__(self, x):
        self.x = x

obj = Class(1)

In [42]:
obj = Class.__new__(Class, 1)
Class.__init__(obj, 1)

In [43]:
X = type("X", (object, ), dict(a=1))

X = type.__new__(type, "X", (object, ), dict(a=1))
type.__init__(X, "X", (object, ), dict(a=1))

type.\_\_new__ возвращает не объект type, а наследник объекта type

In [44]:
class MetaclassB(type):
    def __new__(
        cls: type,
        name: str,
        bases: tuple[type],
        members: dict[str, t.Any]
    ):
        print(f"MetaclassB.__new__(cls={cls}, name={name}, bases={bases}, members={members})")
        return type.__new__(cls, name, bases, members)
        
    def __init__(
        cls: type,
        name: str, 
        bases: tuple[type], 
        members: dict[str, t.Any]
    ):
        print(f"MetaclassB.__init__(cls={cls}, name={name}, bases={bases}, members={members})")
        
class Class4(metaclass=MetaclassB):
    pass

MetaclassB.__new__(cls=<class '__main__.MetaclassB'>, name=Class4, bases=(), members={'__module__': '__main__', '__qualname__': 'Class4'})
MetaclassB.__init__(cls=<class '__main__.Class4'>, name=Class4, bases=(), members={'__module__': '__main__', '__qualname__': 'Class4'})


In [45]:
class MetaclassB(type):
    def __new__(
        cls: type,
        name: str,
        bases: t.Tuple[type],
        members: dict[str, t.Any]
    ):
        print(f"MetaclassB.__new__(cls={cls}, name={name}, bases={bases}, members={members})")
        return type.__new__(cls, name, bases, members)
        
    def __init__(
        cls: type,
        name: str, 
        bases: tuple[type], 
        members: dict[str, t.Any]
    ):
        print(f"MetaclassB.__init__(cls={cls}, name={name}, bases={bases}, members={members})")
        
class Class4(metaclass=MetaclassB):
    pass

MetaclassB.__new__(cls=<class '__main__.MetaclassB'>, name=Class4, bases=(), members={'__module__': '__main__', '__qualname__': 'Class4'})
MetaclassB.__init__(cls=<class '__main__.Class4'>, name=Class4, bases=(), members={'__module__': '__main__', '__qualname__': 'Class4'})


### финалочка

In [46]:
class Metaclass(type):
    pass

class Class(metaclass=Metaclass):
    pass

c = Class()

In [48]:
Class = Metaclass.__new__(Metaclass, "Class", (object, ), dict())
Metaclass.__init__(Class, "Class", (object, ), dict())

c = Class.__new__(Class)
Class.__init__(c)

![all](meta/img/img.004.jpeg "all")

## ABC + abstractmethod

## Настоящий ABC скорее всего сложнее, и нижеследующий наверное не работает в каких-то случаях

__План__
1. Заменить декоратором `@abstractmethod` функции на объекты AbstractMethod
2. В момент создания класса собрать все объекты AbstractMethod в свойство `__abstractmethods__`
3. В момент создания объекта класса проверять пустоту `__abstractmethods__`

In [32]:
class AbstractMethod:
    def __call__(self) -> None:
        raise NotImplementedError("Method not implemented")

def abstractmethod(method: t.Callable[..., t.Any]) -> AbstractMethod:
    return AbstractMethod()

class Animal():
    @abstractmethod
    def hello(self) -> None:
        pass
    
l = Animal()
try:
    l.hello()
except NotImplementedError as e:
    print(e)

Method not implemented


In [29]:
from copy import deepcopy

import inspect


class MyABCMeta(type):
    def __init__(
        cls: type, 
        name: str, 
        bases: tuple[type], 
        dct: dict[str, t.Any]
    ) -> None:
        # Собираем все AbstractMethod из класса, который создаём
        abstract_methods = {name for name, value in dct.items() if isinstance(value, AbstractMethod)}
        # Собираем все AbstractMethod из родителей класса, который создаём
        for base in bases:
            new_methods = inspect.getmembers(base, predicate=lambda x: isinstance(x, AbstractMethod))
            abstract_methods.update({k for k, v in new_methods})
        # Теперь в abstract_methods собрали все методы, которые нужно переписать
        # Собираем все функции, которые есть в классе, который создаём
        concrete_methods = {name for name, value in dct.items() if inspect.isfunction(value)}
        # Записываем все непереопределённые методы в __abstract_methods__
        cls.__abstract_methods__ = abstract_methods - concrete_methods
        
    def __call__(
        cls: type, 
        *args: t.Any,
        **kwargs: t.Any
    ) -> t.Any:
        # Если на момент создания объекта в классе остаются абстрактные методы кидаем ошибку
        if cls.__abstract_methods__:
            methods = ", ".join(cls.__abstract_methods__)
            raise NotImplementedError("Methods not implemented: {}".format(methods))
        return type.__call__(cls, *args, **kwargs)

class MyABC(metaclass=MyABCMeta):
   pass

In [30]:
class Animal(MyABC):
    @abstractmethod
    def hello(self) -> None:
        pass

    
class Cow(Animal):
    def hello(self) -> None:
        print("Moo")
        
class Sheep(Animal):
    pass

In [31]:
try:
    l = Animal()
except NotImplementedError as e:
    print(e)
try:
    s = Sheep()
except NotImplementedError as e:
    print(e)

c = Cow()
c.hello()

Methods not implemented: hello
Methods not implemented: hello
Moo


## Спасибо за внимание
## Вопросы?